In [5]:
import os
import sys
import pandas as pd

sys.path.append("../src")
from pharmgkb import RawData
from utils import CsvCleaner


data_path = os.path.join("..", "data")
pharmgkb_path = os.path.join(data_path, "pharmgkb")
processed_path = os.path.join(data_path, "pharmgkb_processed")

## Quick and dirty exploration of files for curation

### Chemicals

In [ ]:
df = pd.read_csv(os.path.join(pharmgkb_path, "chemicals", "chemicals.csv"))
for i,c in enumerate(df.columns):
    print(i, c)

df["Top FDA Label Testing Level"].unique()

In [ ]:
df[df["Name"]=="apremilast"]

In [ ]:
# Retrieve SMILES for PubChem compounds
import requests

data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/59272813/property/CanonicalSMILES/TXT")
smi = data.text.strip()
smi

In [ ]:
sys.path.append(os.path.join("../src"))

from utils import CsvCleaner
import requests

c = CsvCleaner()

data_dict = {}
for r in df.values:
    chemical = c.stringify(r[1])
    crossr = c.inline_comma_splitter(r[6])
    if crossr is not None:
        for i in crossr:
            print(crossr)
            if len(crossr) == 1:
                if "PubChem Compound" in i:
                    cpd = (i.split(":")[-1])
                    data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/CanonicalSMILES/TXT".format(cpd))
                    smi = data.text.strip()
                    print(smi)
            else:
                if "PubChem Compound" in i:
                    cpd = (i.split(":")[-1][:-1])
                    data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/CanonicalSMILES/TXT".format(cpd))
                    smi = data.text.strip()
                    print(smi)


In [61]:
df = pd.read_csv(os.path.join(processed_path, "chemical.csv"))
print(df.shape)
print(len(df[df["smiles"].isna()]))

(4532, 6)
1756


In [49]:
df[(df["smiles"].isna())&(df["chemical_type"]=="Drug")].head(20)

,cid,chemical,chemical_type,smiles,dosing_guideline,drug_label
179,PA151958637,4-methylthioamphetamine,Drug,NaN,-1,0
180,PA154081778,4-methylumbelliferone,Drug,NaN,-1,0
222,PA151958362,"6,7-dihydroxybergamottin",Drug,NaN,-1,0
270,PA152530740,"8-cyclopentyl-1,3-dipropylxanthine",Drug,NaN,-1,0
293,PA164747080,abatacept,Drug,NaN,-1,0
294,PA448006,abciximab,Drug,NaN,-1,0
298,PA166104276,ABT-751,Drug,NaN,-1,0
313,PA448014,acetamide mea,Drug,NaN,-1,0
315,PA166246281,acetaminophen / caffeine / dihydrocodeine,Drug,NaN,-1,3
320,PA165290927,acetaminophen/propoxyphene napsylate,Drug,NaN,-1,0


In [52]:
data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/adalimumab/property/CanonicalSMILES/TXT")
smi = data.text.strip()
print(smi)

Status: 404
Code: PUGREST.NotFound
Message: No CID found
Detail: No CID found that matches the given name


In [56]:
if "Status: 404" in smi:
    print("yes")

yes


### Genes

In [31]:
df = pd.read_csv(os.path.join(pharmgkb_path, "genes", "genes.csv"),encoding= 'unicode_escape')
for i,c in enumerate(df.columns):
    print(i, c)

0 PharmGKB Accession Id
1 NCBI Gene ID
2 HGNC ID
3 Ensembl Id
4 Name
5 Symbol
6 Alternate Names
7 Alternate Symbols
8 Is VIP
9 Has Variant Annotation
10 Cross-references
11 Has CPIC Dosing Guideline
12 Chromosome
13 Chromosomal Start - GRCh37
14 Chromosomal Stop - GRCh37
15 Chromosomal Start - GRCh38
16 Chromosomal Stop - GRCh38


### Disease

In [33]:
df = pd.read_csv(os.path.join(pharmgkb_path, "phenotypes", "phenotypes.csv"),encoding= 'unicode_escape')
for i,c in enumerate(df.columns):
    print(i, c)

0 PharmGKB Accession Id
1 Name
2 Alternate Names
3 Cross-references
4 External Vocabulary


### Variants

In [10]:
df = pd.read_csv(os.path.join(pharmgkb_path, "variants", "variants.csv"),encoding= 'unicode_escape')
for i,c in enumerate(df.columns):
    print(i, c)

0 Variant ID
1 Variant Name
2 Gene IDs
3 Gene Symbols
4 Location
5 Variant Annotation count
6 Clinical Annotation count
7 Level 1/2 Clinical Annotation count
8 Guideline Annotation count
9 Label Annotation count
10 Synonyms


In [6]:
sys.path.append("../src")
from utils import CsvCleaner
c = CsvCleaner()

# references
vars = pd.read_csv(os.path.join(processed_path, "variant.csv"))
haps = pd.read_csv(os.path.join(processed_path, "haplotype.csv"))
refs = haps["haplotype"].tolist() + vars["variant"].tolist()

df1 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "var_pheno_ann.csv"),encoding= 'unicode_escape')
df2 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "var_drug_ann.csv"),encoding= 'unicode_escape')
df3 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "var_fa_ann.csv"),encoding= 'unicode_escape')
newvars = []
for r in df1.values:
    v = c.inline_comma_splitter_space(r[1])
    for v_ in v:
        newvars += [v_]
for r in df2.values:
    v = c.inline_comma_splitter_space(r[1])
    for v_ in v:
        newvars += [v_]
for r in df3.values:
    v = c.inline_comma_splitter_space(r[1])
    for v_ in v:
        newvars += [v_]

#corrections known
known_haps = ["G6PD Canton", "Taiwan-Hakka", "Gifu-like", "Agrigento-like", "G6PD Mediterranean", "Dallas", "Panama", "Sassari", "Cagliari", "Birmingham"]
newvars = [":".join(value.split(":")[:2]) if value.startswith("HLA-") else value for value in newvars]
newvars = [value for value in newvars if value not in known_haps]
newvars = ["G6PD B (reference)" if value == "G6PD B (wildtype)" else value for value in newvars]
uniquevars = list(set(newvars)-set(refs))
for var in uniquevars:
    if "*" in var: #this is not a variant but a haplotype, will be added manually to haplotypes.csv
        uniquevars.remove(var)
        print(var)

df = pd.DataFrame(uniquevars, columns=["variant"])
df.to_csv(os.path.join(processed_path, "orphan_variant_list.csv"), index=False)

CYP3A4*36


### Haplotypes

In [29]:
df = pd.read_csv(os.path.join(pharmgkb_path, "relationships", "relationships.csv"),encoding= 'unicode_escape')
for i,c in enumerate(df.columns):
    print(i, c)

0 Entity1_id
1 Entity1_name
2 Entity1_type
3 Entity2_id
4 Entity2_name
5 Entity2_type
6 Evidence
7 Association
8 PK
9 PD
10 PMIDs


In [ ]:
#We keep all the haplotypes that appear, either in entity 1 or in entity 2, and then get the gene from the Haplotype name
df1 = df["Entity1_name"][df["Entity1_type"]=="Haplotype"].tolist()
print(len(df1))
df2 = df["Entity2_name"][df["Entity2_type"]=="Haplotype"].tolist()
print(len(df2))
print(len(set(df1+df2)))

#get unique hids for either entity 1 or entity2 -- is the same
haps = list(set(df1+df2))
hids = []
for r in df.values:
    hap = r[1]
    hid = r[0]
    if hap in haps:
        hids += [hid]
len(hids)
len(set(hids))

haps = list(set(df1+df2))
hids = []
for r in df.values:
    hap = r[4]
    hid = r[3]
    if hap in haps:
        hids += [hid]
len(hids)
len(set(hids))
hids = list(set(hids))

In [34]:
#We keep all the haplotypes that appear, either in entity 1 or in entity 2, and then get the gene from the Haplotype name
df1 = df[df["Entity1_type"]=="Haplotype"]
print(df1.shape)
df1.drop_duplicates(subset= ["Entity1_name"], keep="first", inplace=True)
print(df1.shape)
df1 = df1[["Entity1_id", "Entity1_name"]]
df1.rename(columns={"Entity1_id": "hid", "Entity1_name":"haplotype"}, inplace=True)

(11788, 11)
(1028, 11)


/tmp/ipykernel_10338/925833839.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop_duplicates(subset= ["Entity1_name"], keep="first", inplace=True)


In [47]:
gene = []
for h in df1["haplotype"]:
    if " " in h:
        g = h.split(" ")[0]
    elif "*" in h:
        g = h.split("*")[0]
    else:
        g = h
    gene += [g]
df1["gene"]= gene
gl = list(set(df1["gene"].tolist()))
gl

### Clincal variants

In [ ]:
df = pd.read_csv(os.path.join(pharmgkb_path, "clinicalVariants", "clinicalVariants.csv"),encoding= 'unicode_escape')
for i,c in enumerate(df.columns):
    print(i, c)

### Clinical Annotations

In [60]:
#clinical annotations are identified by a single ID but can be  referenced in different files
al = pd.read_csv(os.path.join(pharmgkb_path, "clinicalAnnotations", "clinical_ann_alleles.csv"))
ev = pd.read_csv(os.path.join(pharmgkb_path, "clinicalAnnotations", "clinical_ann_evidence.csv"),encoding= 'unicode_escape')
his = pd.read_csv(os.path.join(pharmgkb_path, "clinicalAnnotations", "clinical_ann_history.csv"),encoding= 'unicode_escape')
cl = pd.read_csv(os.path.join(pharmgkb_path, "clinicalAnnotations", "clinical_annotations.csv"),encoding= 'unicode_escape')


print("Alleles")
for i,c in enumerate(al.columns):
    print(i, c)
print("Evidence")
for i,c in enumerate(ev.columns):
    print(i, c)
print("History")
for i,c in enumerate(his.columns):
    print(i, c)
print("ClinicalAnn")
for i,c in enumerate(cl.columns):
    print(i, c)

Alleles
0 Clinical Annotation ID
1 Genotype/Allele
2 Annotation Text
3 Allele Function
4 test
Evidence
0 Clinical Annotation ID
1 Evidence ID
2 Evidence Type
3 Evidence URL
4 PMID
5 Summary
6 Score
History
0 Clinical Annotation ID
1 Date (YYYY-MM-DD)
2 Type
3 Comment
ClinicalAnn
0 Clinical Annotation ID
1 Variant/Haplotypes
2 Gene
3 Level of Evidence
4 Level Override
5 Level Modifiers
6 Score
7 Phenotype Category
8 PMID Count
9 Evidence Count
10 Drug(s)
11 Phenotype(s)
12 Latest History Date (YYYY-MM-DD)
13 URL
14 Specialty Population


In [70]:
#Use a CAID (Clinical Ann ID) as an example see the information across files
caid = 1183615480
al[al["Clinical Annotation ID"]==caid]

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,test
2809,1183615480,*1,Patients carrying the CYP2D6*1 allele in combi...,Normal function,NaN
2810,1183615480,*2,Patients carrying the CYP2D6*2 allele in combi...,Normal function,NaN
2811,1183615480,*3,Patients carrying the CYP2D6*3 allele in combi...,No function,NaN
2812,1183615480,*4,Patients carrying the CYP2D6*4 allele in combi...,No function,NaN
2813,1183615480,*5,Patients carrying the CYP2D6*5 allele in combi...,No function,NaN
2814,1183615480,*6,Patients carrying the CYP2D6*6 allele in combi...,No function,NaN
2815,1183615480,*10,Patients carrying the CYP2D6*10 allele in comb...,Decreased function (AV 0.25),NaN


In [71]:
ev[ev["Clinical Annotation ID"]==caid]

,Clinical Annotation ID,Evidence ID,Evidence Type,Evidence URL,PMID,Summary,Score
2765,1183615480,827807126,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/827...,21599570.0,CYP2D6 *1/*1 is associated with increased clea...,0
2766,1183615480,827807118,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/827...,21599570.0,CYP2D6 *1/*1 is associated with increased clea...,0
2767,1183615480,827807123,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/827...,21599570.0,CYP2D6 *1/*1 is associated with increased clea...,0
2768,1183615480,827807099,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/827...,21599570.0,CYP2D6 *1/*1 is associated with increased clea...,0
2769,1183615480,982036882,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/982...,16849011.0,CYP2D6 *10 is associated with decreased metabo...,1
2770,1183615480,982036911,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/982...,16849011.0,CYP2D6 *4 is not associated with metabolism of...,-0.125
2771,1183615480,982036924,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/982...,16849011.0,CYP2D6 *5 is not associated with metabolism of...,-0.125
2772,1183615480,982043452,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/982...,16595916.0,CYP2D6 *10/*10 is associated with decreased cl...,2.25
2773,1183615480,982044314,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/982...,17329852.0,CYP2D6 *1/*4 is associated with decreased clea...,1.5
2774,1183615480,982044321,Variant Drug Annotation,https://www.pharmgkb.org/variantAnnotation/982...,17329852.0,CYP2D6 *1/*5 is associated with decreased clea...,1.5


In [72]:
cl[cl["Clinical Annotation ID"]==caid]

,Clinical Annotation ID,Variant/Haplotypes,Gene,Level of Evidence,Level Override,Level Modifiers,Score,Phenotype Category,PMID Count,Evidence Count,Drug(s),Phenotype(s),Latest History Date (YYYY-MM-DD),URL,Specialty Population
875,1183615480,"CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6...",CYP2D6,3,NaN,Tier 1 VIP,7.5,Metabolism/PK,4,11,carvedilol,Heart Diseases,22/04/2021,https://www.pharmgkb.org/clinicalAnnotation/11...,NaN


In [66]:
his[his["Clinical Annotation ID"]==caid]

,Clinical Annotation ID,Date (YYYY-MM-DD),Type,Comment
1989,1183615480,18/02/2017,Update,NaN
1990,1183615480,12/09/2018,Update,Removed inaccurate variant annotation on PMID ...
1991,1183615480,05/11/2018,Update,Updated OMB race to appropriate biogeographica...
1992,1183615480,05/02/2021,Update,Added PMIDs 16849011 and 21599570 to evidence....
1993,1183615480,24/03/2021,Update,CA score added as part of scoring system relea...
1994,1183615480,22/04/2021,Update,Added sentence about DPWG 'no recommendation' ...


In [50]:
#create temporal smaller files to work on:

df = pd.read_csv(os.path.join(processed_path, "clinical_annotation.csv"))
df = df.head(1000)
df.to_csv(os.path.join(processed_path, "clinann_tmp.csv"), index=False)

df = pd.read_csv(os.path.join(pharmgkb_path, "clinicalAnnotations", "clinical_ann_alleles.csv"))
df = df.head(1000)
df.to_csv(os.path.join(processed_path, "clinann_all_tmp.csv"), index=False)

#### Clinical variant vs clinical Annotation

In [3]:
df1 = pd.read_csv(os.path.join(processed_path, "clinical_annotation.csv"))
df2 = pd.read_csv(os.path.join(processed_path, "clinical_variant.csv"))

print(df1.columns)
print(df2.columns)

Index(['caid', 'vid', 'variant', 'hid', 'haplotype', 'gene', 'evidence',
       'score', 'phenotype', 'chemical', 'disease'],
      dtype='object')
Index(['vid', 'variant', 'hid', 'haplotype', 'gene', 'phenotype', 'evidence',
       'chemical', 'disease'],
      dtype='object')


In [7]:
# Select the columns to compare for duplication
columns_to_compare = ['vid', 'variant', 'hid', 'haplotype', 'gene', 'chemical', "evidence"]

# Find duplicated rows in df1 based on the selected columns
duplicated_rows_df1 = df1[df1.duplicated(subset=columns_to_compare, keep=False)]

# Find duplicated rows in df2 based on the selected columns
duplicated_rows_df2 = df2[df2.duplicated(subset=columns_to_compare, keep=False)]

# Combine the duplicated rows from both dataframes
duplicated_rows = pd.concat([duplicated_rows_df1, duplicated_rows_df2], ignore_index=True)

# Print the duplicated rows
print(len(duplicated_rows))


11533


#### Variant Annotations

In [12]:
df1 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "study_parameters.csv"))
df2 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "var_drug_ann.csv"))
df3 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "var_fa_ann.csv"))
df4 = pd.read_csv(os.path.join(pharmgkb_path, "variantAnnotations", "var_pheno_ann.csv"), encoding= 'unicode_escape')

print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)

Index(['Study Parameters ID', 'Variant Annotation ID', 'Study Type',
       'Study Cases', 'Study Controls', 'Characteristics',
       'Characteristics Type', 'Frequency In Cases',
       'Allele Of Frequency In Cases', 'Frequency In Controls',
       'Allele Of Frequency In Controls', 'P Value', 'Ratio Stat Type',
       'Ratio Stat', 'Confidence Interval Start', 'Confidence Interval Stop',
       'Biogeographical Groups'],
      dtype='object')
Index(['Variant Annotation ID', 'Variant/Haplotypes', 'Gene', 'Drug(s)',
       'PMID', 'Phenotype Category', 'Significance', 'Notes', 'Sentence',
       'Alleles', 'Specialty Population'],
      dtype='object')
Index(['Variant Annotation ID', 'Variant/Haplotypes', 'Gene', 'Drug(s)',
       'PMID', 'Phenotype Category', 'Significance', 'Notes', 'Sentence',
       'Alleles', 'Specialty Population'],
      dtype='object')
Index(['Variant Annotation ID', 'Variant/Haplotypes', 'Gene', 'Drug(s)',
       'PMID', 'Phenotype Category', 'Significance',

In [13]:
#How do the different variant IDs relate to each other
col = "Variant Annotation ID"

print(len(df1), len(df2), len(df3), len(df4))
print(len(set(df1[col])))
print(len(set(df1[col]) & set(df2[col])))
print(len(set(df1[col]) & set(df3[col])))
print(len(set(df1[col]) & set(df4[col])))
print(len(set(df2[col]) & set(df4[col])))

32076 11222 1810 12844
24993
11104
1157
12732
0


In [14]:
#Are Variant Annotation IDs different to Variant IDs? YES
var = pd.read_csv(os.path.join(processed_path, "variant.csv"))
print(len(set(df1[col]) & set(var["vid"])))


0


In [15]:
#Are Variant Annotation IDs different to Clinical Annotation IDs? YES
clan = pd.read_csv(os.path.join(processed_path, "clinical_annotation_allele.csv"))
print(len(set(df1[col]) & set(clan["caid"])))

0


In [20]:
#Can we append the bioG group and study id to the var_drug_ann and var_pheno_ann files using vaid?
df_ = pd.read_csv(os.path.join(processed_path, "var_drug_ann.csv"))
print(len(set(df_["vaid"])))
print(len(list(set(df1["Variant Annotation ID"])&set(df_["vaid"]))))

10471
10359


In [18]:
#Compare Variant Annotations and Clinical Annotations
clan[(clan["gene"]=="ABCB1")&(clan["chemical"]=="efavirenz")]

,caid,vid,variant,hid,haplotype,gene,evidence,score,phenotype,chemical,disease,allele,anntext,function
7527,655385701,PA166157284,rs1045642,NaN,NaN,ABCB1,3,5.750,Efficacy,efavirenz,HIV Infections,AA,Patients with the AA genotype and HIV who are ...,NaN
7528,655385701,PA166157284,rs1045642,NaN,NaN,ABCB1,3,5.750,Efficacy,efavirenz,HIV Infections,AG,Patients with the GA genotype and HIV who are ...,NaN
7529,655385701,PA166157284,rs1045642,NaN,NaN,ABCB1,3,5.750,Efficacy,efavirenz,HIV Infections,GG,Patients with the GG genotype and HIV who are ...,NaN
7530,655385701,PA166157284,rs1045642,NaN,NaN,ABCB1,3,5.750,Toxicity,efavirenz,HIV Infections,AA,Patients with the AA genotype and HIV who are ...,NaN
7531,655385701,PA166157284,rs1045642,NaN,NaN,ABCB1,3,5.750,Toxicity,efavirenz,HIV Infections,AG,Patients with the GA genotype and HIV who are ...,NaN
7532,655385701,PA166157284,rs1045642,NaN,NaN,ABCB1,3,5.750,Toxicity,efavirenz,HIV Infections,GG,Patients with the GG genotype and HIV who are ...,NaN
10161,981238288,PA166157316,rs2032582,NaN,NaN,ABCB1,4,-0.250,Efficacy,efavirenz,HIV Infections,AA,Patients with the AA genotype may have decreas...,NaN
10162,981238288,PA166157316,rs2032582,NaN,NaN,ABCB1,4,-0.250,Efficacy,efavirenz,HIV Infections,AC,Patients with the AC genotype may have decreas...,NaN
10163,981238288,PA166157316,rs2032582,NaN,NaN,ABCB1,4,-0.250,Efficacy,efavirenz,HIV Infections,CC,Patients with the CC genotype may have increas...,NaN
13455,1184233711,PA166157284,rs1045642,NaN,NaN,ABCB1,3,0.500,Other,efavirenz,HIV Infections,AA,Patients with the AA genotype and HIV infectio...,NaN


In [20]:
df4[(df4["Gene"]=="ABCB1")&(df4["Drug(s)"]=="efavirenz")]

,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population
13,655385693,rs1045642,ABCB1,efavirenz,16267764,Other,no,"Although not significant, a trend for increasi...",Genotype AA is associated with increased CD4 T...,AA,NaN
14,655385696,rs1045642,ABCB1,efavirenz,16267764,Efficacy,yes,Please note; 3435C > T alleles were given in t...,Genotype AA is associated with increased likel...,AA,NaN
1407,1184168552,rs1045642,ABCB1,efavirenz,23734829,Toxicity,no,No association was seen between the alleles of...,Allele G is not associated with Neurotoxicity ...,G,NaN
4032,827695550,rs2032582,ABCB1,efavirenz,16267764,Metabolism/PK,no,"Genotypes GG, GT, TT, AG, AT (minus strand) we...",Genotypes AT + CT are not associated with incr...,AT + CT,NaN
5973,827695530,rs1045642,ABCB1,efavirenz,16267764,Metabolism/PK,no,Please note; 3435C > T alleles were given in t...,Genotype AA is not associated with increased l...,AA,NaN
6061,827695457,rs1045642,ABCB1,efavirenz,16267764,Toxicity,yes,Please note; 3435C > T alleles were given in t...,Allele A is associated with increased likeliho...,A,NaN
8752,1448993831,rs1045642,ABCB1,efavirenz,26715213,Toxicity,yes,NaN,Genotype AA are associated with increased risk...,AA,NaN
10657,769163058,rs3842,ABCB1,efavirenz,20860463,Metabolism/PK,yes,PBMC accumulation was calculated by: (cell con...,Genotypes CC + CT are associated with increase...,CC + CT,NaN
10659,769163062,rs1128503,ABCB1,efavirenz,20860463,Metabolism/PK,no,No association with accumulation of efavirenz ...,Genotype AA is not associated with increased m...,AA,NaN
10662,769163063,rs2032582,ABCB1,efavirenz,20860463,Metabolism/PK,no,No association was found with efavirenz accumu...,Genotype AA is not associated with increased m...,AA,NaN


### Haplotype Deconvolution
For which genes is there information about the haplotype? Consider all that appear in:
- Clinical Annotations
- Variant Annotations
- Haplotype

In [2]:
df1 = pd.read_csv(os.path.join(processed_path, "clinical_annotation.csv"))
df2 = pd.read_csv(os.path.join(processed_path, "clinical_variant.csv"))
df3 = pd.read_csv(os.path.join(processed_path, "var_drug_ann.csv"))
df4 = pd.read_csv(os.path.join(processed_path, "var_pheno_ann.csv"))
df5 = pd.read_csv(os.path.join(processed_path, "haplotype.csv"))
genes = (df1["gene"].tolist()+df2["gene"].tolist()+df3["gene"].tolist()+df4["gene"].tolist()+df5["gene"].tolist())

In [12]:
gene_names = list(
    set(
        df1["gene"][~df1["haplotype"].isna()].tolist()
        + df2["gene"][~df2["haplotype"].isna()].tolist()
        + df3["gene"][~df3["haplotype"].isna()].tolist()
        + df4["gene"][~df4["haplotype"].isna()].tolist()
        + df5["gene"].tolist()
    )
)

In [26]:
df = pd.read_csv(os.path.join(processed_path, "haplotypes_gene", "G6PD_haplotypes.csv"))
df["haplotype"][df["hid"].isna()].tolist()

['G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (reference)',
 'G6PD B (r

#### Drug Labels

In [28]:
df = pd.read_csv(os.path.join(pharmgkb_path, "drugLabels", "drugLabels.csv"))
df.columns

Index(['PharmGKB ID', 'Name', 'Source', 'Biomarker Flag', 'Testing Level',
       'Has Prescribing Info', 'Has Dosing Info', 'Has Alternate Drug',
       'Cancer Genome', 'Prescribing', 'Chemicals', 'Genes',
       'Variants/Haplotypes', 'Latest History Date (YYYY-MM-DD)'],
      dtype='object')

#### Automated annotations

In [4]:
df = pd.read_csv(os.path.join(pharmgkb_path, "automated_annotations", "automated_annotations.csv"))
df.columns

Index(['Chemical ID', 'Chemical Name', 'Chemical in Text', 'Variation ID',
       'Variation Name', 'Variation Type', 'Variation in Text', 'Gene IDs',
       'Gene Symbols', 'Gene in Text', 'Literature ID', 'PMID',
       'Literature Title', 'Publication Year', 'Journal', 'Sentence',
       'Source'],
      dtype='object')

In [6]:
#check how many do not have gene info
print(df.shape)
print(df[~df["Gene Symbols"].isna()].shape)

(20223, 17)
(12865, 17)


In [8]:
df[df["Gene Symbols"].isna()].head(20)

,Chemical ID,Chemical Name,Chemical in Text,Variation ID,Variation Name,Variation Type,Variation in Text,Gene IDs,Gene Symbols,Gene in Text,Literature ID,PMID,Literature Title,Publication Year,Journal,Sentence,Source
0,PA449028,clarithromycin,clarithromycin,NaN,NaN,Variant,A2146G,NaN,NaN,NaN,NaN,31528091,Review of current diagnostic methods and advan...,2019,World Journal of Gastroenterology,"Moreover, they can distinguish the three most ...",PGxMine
5,PA448515,azathioprine,azathioprine,rs10889677,rs10889677,Variant,rs10889677,NaN,NaN,NaN,NaN,28210080,IL23R single nucleotide polymorphisms could b...,2017,World Journal of Gastroenterology,"For patients with the rs10889677, a therapy wi...",PGxMine
6,PA448515,azathioprine,azathioprine,rs10889677,rs10889677,Variant,rs10889677,NaN,NaN,NaN,NaN,28210080,IL23R single nucleotide polymorphisms could b...,2017,World Journal of Gastroenterology,The rs10889677 was associated with azathioprin...,PGxMine
7,PA448515,azathioprine,azathioprine,rs10889677,rs10889677,Variant,rs10889677,NaN,NaN,NaN,NaN,28210080,IL23R single nucleotide polymorphisms could b...,2017,World Journal of Gastroenterology,The rs10889677 was associated with azathioprin...,PGxMine
8,PA452639,infliximab,infliximab,rs2522057,rs2522057,Variant,rs2522057,NaN,NaN,NaN,NaN,28785150,Genetic polymorphisms predict response to anti...,2017,World Journal of Gastroenterology,A SNP in this cluster (rs2522057) may be assoc...,PGxMine
10,PA166122590,simeprevir,simeprevir,NaN,NaN,Variant,Q80K,NaN,NaN,NaN,NaN,27920481,New real-time-PCR method to identify single po...,2016,World Journal of Gastroenterology,One such naturally occurring mutation is the Q...,PGxMine
11,PA166122590,simeprevir,simeprevir,NaN,NaN,Variant,Q80K,NaN,NaN,NaN,NaN,27920481,New real-time-PCR method to identify single po...,2016,World Journal of Gastroenterology,One well-documented example is the Q80K polymo...,PGxMine
12,PA166128167,daclatasvir,daclatasvir,NaN,NaN,Variant,Y93H,NaN,NaN,NaN,NaN,27920481,New real-time-PCR method to identify single po...,2016,World Journal of Gastroenterology,We are now working on adapting this method for...,PGxMine
13,PA166122590,simeprevir,simeprevir,NaN,NaN,Variant,Q80K,NaN,NaN,NaN,NaN,27920481,New real-time-PCR method to identify single po...,2016,World Journal of Gastroenterology,"In addition, Q80K seems to be a natural varian...",PGxMine
14,PA449028,clarithromycin,clarithromycin,NaN,NaN,Variant,A2142G,NaN,NaN,NaN,NaN,28465629,Noninvasive molecular analysis of Helicobacter...,2017,World Journal of Gastroenterology,A first noninvasive investigation using stool ...,PGxMine
